In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from skmultilearn.model_selection import iterative_train_test_split

from imblearn.over_sampling import RandomOverSampler

import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv("database.csv") 
mapping = {'L': 0, 'M': 1, 'H': 2}
df['Type'] = df['Type'].map(mapping)

X = df[['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]"]] 
Y = df[["TWF", "HDF", "PWF", "OSF", "RNF"]] 
df_twf = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_hdf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_pwf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_osf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_rnf = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 1)]
print(len(df_twf), len(df_hdf), len(df_pwf), len(df_osf), len(df_rnf))


42 106 80 78 18


In [ ]:
df_2to = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_2tr = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 1)]

df_2hp = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 0) & 
          (df["RNF"] == 0)]

df_2ho = df[(df["TWF"] == 0) & 
          (df["HDF"] == 1) & 
          (df["PWF"] == 0) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]

df_2po = df[(df["TWF"] == 0) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]


print(len(df_2to), len(df_2tr), len(df_2hp), len(df_2ho), len(df_2po))

In [ ]:
df_3 = df[(df["TWF"] == 1) & 
          (df["HDF"] == 0) & 
          (df["PWF"] == 1) & 
          (df["OSF"] == 1) & 
          (df["RNF"] == 0)]
print(len(df_3))

In [21]:
X_np = X.values
Y_np = Y.values

X_train, Y_train, X_test, Y_test = iterative_train_test_split(X_np, Y_np, test_size=0.3)

In [23]:
n_muestras_a_agregar = 50
X_min=df[df["RNF"]==1]
X_minoritaria=X_min[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', "Tool wear [min]"]] 
y_minoritaria=Y[Y['RNF']==1]
rng = np.random.default_rng(seed=42)
# Realizar sobremuestreo aleatorio
indices_aleatorios = rng.integers(0, len(X_minoritaria), size=n_muestras_a_agregar)
X_minoritaria_sobremuestreada = X_minoritaria.iloc[indices_aleatorios]
y_minoritaria_sobremuestreada =y_minoritaria.iloc[indices_aleatorios]
X_sobremuestreado = pd.concat([X, X_minoritaria_sobremuestreada], ignore_index=True)
y_sobremuestreado = pd.concat([Y, y_minoritaria_sobremuestreada], ignore_index=True)
X=X_sobremuestreado
Y=y_sobremuestreado


In [22]:
#modelo de prediccion
labels = ["TWF", "HDF", "PWF", "OSF", "RNF"]
rf = RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42) 
multi_rf = MultiOutputClassifier(rf) 
multi_rf.fit(X_train, Y_train) 
# Evaluacion 
Y_pred = multi_rf.predict(X_test) 
print(classification_report(Y_test, Y_pred, target_names=labels))

              precision    recall  f1-score   support

         TWF       1.00      0.13      0.24        15
         HDF       0.92      0.63      0.75        35
         PWF       0.83      0.54      0.65        28
         OSF       1.00      0.62      0.77        29
         RNF       1.00      0.69      0.81        32

   micro avg       0.94      0.57      0.71       139
   macro avg       0.95      0.52      0.64       139
weighted avg       0.95      0.57      0.69       139
 samples avg       0.02      0.02      0.02       139



d:\Katy\Instalados\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Katy\Instalados\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Katy\Instalados\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
oversampler = sv.ML_SMOTE() # crea el generador multilabel X_res_np, Y_res_np = oversampler.sample(X_train_np, Y_train_np) # 5) Volver a DataFrame X_train_res = pd.DataFrame(X_res_np, columns=X_train.columns) Y_train_res = pd.DataFrame(Y_res_np.astype(int), columns=Y_train.columns) # 6) Entrenar modelo multietiqueta (ejemplo RandomForest) rf = RandomForestClassifier(n_estimators=200, random_state=42) multi_rf = MultiOutputClassifier(rf) multi_rf.fit(X_train_res, Y_train_res) # 7) Evaluación en test (sin sobremuestreo) Y_pred = multi_rf.predict(X_test) print(classification_report(Y_test, Y_pred, target_names=label_cols))